1. Estimation of emission function
2. Transform from short to long first, keep the 'generation year' variable 
3. Predict emissions in the sales dataset base on the generation year variable
4. Aggregate trims
    use sales-weighted average!
5. Add market size
6. Construct instrument
7. Add demographics
8. Convert currency to GBP and construct subsidized prices
9. Run BLP
    9.1. match EV range values
    9.2. match emission values

Data cleaning:
1. Annual sales = 0
2. gross vehicle weight < 0
3. Unspecified price
3. luxury cars  (price > £150,000)
4. Niche trim (e.g. annual sales below 100)


To-do-lists:
1. Find the best specification without controlling for emissions and EV ranges
    1.1. add charging data for UK, France, and Germany from 2015 to 2023

2. try your best to reconcide variable names in both sales and emission datasets.
    2.1.  also reconcile values of variables (e.g. fuel type)

3. Remove duplicates in the emission functions and replace 'r' with number of replicates

4. Estimate Emission function
    4.1. add more controls
    4.2. separately for NEDC and WLTP
5. Write codes to do vague matching

In [ ]:
# update PyBLP’s Anaconda dependencies
conda update numpy scipy sympy patsy
# install PyBLP
pip install pyblp

In [4]:
import pyblp
import pandas as pd
import numpy as np
pd.set_option("display.max_columns", None)  # print all columns of a dataframe; we can replace "None" by any number if we want to print a certain number of columns
pd.set_option("max_colwidth", None)   # Set the Column Width.
pd.set_option("display.max_rows", 100) 
np.set_printoptions(precision=3)
pd.options.display.float_format
pyblp.options.digits = 2
pyblp.options.verbose = False
pyblp.__version__

'1.1.0'

# Load all data (4min30s)

In [2]:
################################################################################          Windows          ################################################################################
### Load emission data
#df_emission = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Official fuel efficiency/data/2010-2023_fe_uk.csv", low_memory=False) 

### Load Car Sales data
df_uk_sales = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/Sales/2024_enhanced_uk.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_fr_sales = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/Sales/2024_enhanced_fr.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_ge_sales = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/Sales/2024_enhanced_ge.csv', encoding='unicode_escape',  engine='python')
# df_ge_sales = pd.read_csv('C:/Users/uctpql0/OneDrive - University College London/Cars data/Sales/2024_enhanced_ge.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')   # this way of loading deletes some rows with errors, leading to only 286562 rows

# rename the columns 'Country/Territory-Name' or 'ï»¿c' to 'Country'
df_uk_sales.rename(columns={'Country/Territory-Name':'Country'}, inplace=True)
df_fr_sales.rename(columns={'Country/Territory-Name':'Country'}, inplace=True)
df_ge_sales.rename(columns={'ï»¿c':'Country'}, inplace=True)

# concatenate the three dataframes horizontally
df_sales = pd.concat([df_uk_sales, df_fr_sales, df_ge_sales],axis=0, ignore_index=True)

### load household number data
df_size = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/number of household_uk_ge_fr.csv")

### Load Pound-Euro exchange rate
df_rate = pd.read_excel("C:/Users/uctpql0/OneDrive - University College London/Cars data/exchange rate.xlsx")

### add charing devices data
df_charging = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/charging infrastructure/Charging points_uk_fr_ge.csv")

### add gdp/capita data
df_gdp = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/gdp_pc.csv")

### add EV stock data
df_evstock = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/ev_stock.csv")
### Load Demographic Survey data
#df_dm22 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd22_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm21 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd21_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm20 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd20_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm19 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd19_eul.tab",sep='\t',low_memory=False)
#df_dm18 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd18_eul.tab",sep='\t',low_memory=False)
#df_dm17 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd17_eul.tab",sep='\t',low_memory=False)
#df_dm16 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd16_eul.tab",sep='\t',low_memory=False)
#df_dm15 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd15_eul.tab",sep='\t',low_memory=False)
#df_dm14 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd14_eul.tab",sep='\t',low_memory=False)
#df_dm13 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd13_eul.tab",sep='\t',low_memory=False)
#df_dm12 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd12_eul.tab",sep='\t',low_memory=False)
#df_dm11 = pd.read_csv("C:/Users/uctpql0/OneDrive - University College London/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd11_eul.tab",sep='\t',low_memory=False)
### Add a market_ids column to each dataframe
#df_dm22['market_ids'] = '2022'
#df_dm21['market_ids'] = '2021'
#df_dm20['market_ids'] = '2020'
#df_dm19['market_ids'] = '2019'
#df_dm18['market_ids'] = '2018'
#df_dm17['market_ids'] = '2017'
#df_dm16['market_ids'] = '2016'
#df_dm15['market_ids'] = '2015'
#df_dm14['market_ids'] = '2014'
#df_dm13['market_ids'] = '2013'
#df_dm12['market_ids'] = '2012'
#df_dm11['market_ids'] = '2011'

In [2]:
################################################################################ Mac IOS ################################################################################
### Load emission data
#df_emission = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Official fuel efficiency/data/2010-2023_fe_uk.csv',low_memory=False) 
### Load Car Sales data
df_uk_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Sales/2024_enhanced_uk.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_fr_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Sales/2024_enhanced_fr.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
df_ge_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Sales/2024_enhanced_ge.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
# rename the columns 'Country/Territory-Name' or 'ï»¿c' to 'Country'
df_uk_sales.rename(columns={'Country/Territory-Name':'Country'}, inplace=True)
df_fr_sales.rename(columns={'Country/Territory-Name':'Country'}, inplace=True)
df_ge_sales.rename(columns={'ï»¿c':'Country'}, inplace=True)
# concatenate the three dataframes horizontally
df_sales = pd.concat([df_uk_sales, df_fr_sales, df_ge_sales],axis=0, ignore_index=True)
#df_sales = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Sales/2024_enhanced_uk_fr_ge.csv', encoding='unicode_escape',  engine='python', sep=None,  quoting=3, on_bad_lines='skip')
### load household number data
df_size = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/number of household_uk_ge_fr.csv")
### Load Pound-Euro exchange rate
#df_rate = pd.read_excel("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/exchange rate.xlsx")
### add charing devices data
df_charging = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/charging infrastructure/Charging points_uk_fr_ge.csv")
### add gdp/capita data
df_gdp = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/gdp_pc.csv")
### add EV stock data
df_evstock = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/ev_stock.csv")
### Load Demographic Survey data
#df_dm22 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd22_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm21 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd21_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm20 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd20_eul_phhwta22.tab",sep='\t',low_memory=False)
#df_dm19 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd19_eul.tab",sep='\t',low_memory=False)
#df_dm18 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd18_eul.tab",sep='\t',low_memory=False)
#df_dm17 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd17_eul.tab",sep='\t',low_memory=False)
#df_dm16 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd16_eul.tab",sep='\t',low_memory=False)
#df_dm15 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd15_eul.tab",sep='\t',low_memory=False)
#df_dm14 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd14_eul.tab",sep='\t',low_memory=False)
#df_dm13 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd13_eul.tab",sep='\t',low_memory=False)
#df_dm12 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd12_eul.tab",sep='\t',low_memory=False)
#df_dm11 = pd.read_csv("/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Consumer demographics/UK/UK Annual Household Survey/tab files/apsh_jd11_eul.tab",sep='\t',low_memory=False)
### Add a market_ids column to each dataframe
#df_dm22['market_ids'] = '2022'
#df_dm21['market_ids'] = '2021'
#df_dm20['market_ids'] = '2020'
#df_dm19['market_ids'] = '2019'
#df_dm18['market_ids'] = '2018'
#df_dm17['market_ids'] = '2017'
#df_dm16['market_ids'] = '2016'
#df_dm15['market_ids'] = '2015'
#df_dm14['market_ids'] = '2014'
#df_dm13['market_ids'] = '2013'
#df_dm12['market_ids'] = '2012'
#df_dm11['market_ids'] = '2011'

# 1. Estimate the Emission function   (60s)

In [ ]:
#df_emission = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Official fuel efficiency/data/2010-2023_fe_uk.csv',low_memory=False) 
de = df_emission.copy()

In [ ]:
### adding a 'e_year' column that is the earliest year of the model being sold. 
# n.b. for models that are sold earlier than 2010, we assume that they were first sold in 2010.
de['e_year'] = de.groupby(['va', 've', 'cn'])['year'].transform('min')
de = de[~de['e_year'].isna()]   # some models have missing values in 'e_year' due to NaN in 'va', 've', 'cn', we remove them
de['e_year'] = de['e_year'].astype(int)

### Rename the columns names
de.columns = de.columns.str.lower()  # convert column names to lower case
de.rename(columns={'mh':'firm','mk':'brand','m (kg)':'weight', 'ec (cm3)':'power','ep (kw)':'horsepower','enedc (g/km)':'nedc', 'ewltp (g/km)':'wltp'}, inplace=True)  # rename columns

### Rename fuel type for estimation
de['ft'] = de['ft'].str.lower()
de['ft'] = de['ft'].str.rstrip()
de['ft'] = de['ft'].str.lstrip()
de['ft'] = de['ft'].replace('lpg','alternative_fuel')
de['ft'] = de['ft'].replace('ng-biomethane','alternative_fuel')
de['ft'] = de['ft'].replace('ng_biomethane','alternative_fuel')
de['ft'] = de['ft'].replace('e85','alternative_fuel')
de['ft'] = de['ft'].replace('biodiesel','alternative_fuel')
de['ft'] = de['ft'].replace('petrol-electric','petrol_phev')
de['ft'] = de['ft'].replace('hydrogen','alternative_fuel')
de['ft'] = de['ft'].replace('diesel-electric','diesel_phev')
de['ft'] = de['ft'].replace('petrol-gas','alternative_fuel')
de['ft'] = de['ft'].replace('hybrid/petrol/e','petrol')
de['ft'] = de['ft'].replace('petrol phev','petrol_phev')
de['ft'] = de['ft'].replace('petrol/electric','petrol_phev')
de['ft'] = de['ft'].replace('diesel/electric','diesel_phev')
de['ft'] = de['ft'].replace('ng','alternative_fuel')    # natural gas
de['ft'] = de['ft'].replace('cng','alternative_fuel')    # compressed natural gas
de['ft'] = de['ft'].replace('gnl','petrol')    # gasoline

### Remove duplicates and keep the first occurrence of duplicates (goal: give more equal weights between earlier and later periods)
de = de.drop(columns=['id']) # remove the 'id' columns
de_no_duplicated = de.drop_duplicates()  # remove duplicated rows

### Remove missing values
de = de.dropna(subset=['power'])    # drop missing values in 'power'
de = de.dropna(subset=['nedc'])     # drop missing values in 'nedc'
de = de.dropna(subset=['weight'])   # drop missing values in 'weight'
de = de.dropna(subset=['ft'])      # drop missing values in 'ft'

### Remove certain fuel types
de = de[de['ft']!='unknown']
de = de[de['ft']!='']
de = de[de['ft']!='other']
de = de[de['ft']!='electric']

### Generate dummies for categorical variables, i.e. fuel type and years
de_dummies = pd.get_dummies(de[['ft','e_year']], columns=['ft', 'e_year'], drop_first=False)  # One-hot encode the columns 'ft' and 'year'
de_dummies = df_emission_dummies.astype(int)    # Convert the encoded columns to integer type
de_encoded = pd.concat([de, de_dummies], axis=1)    # Concatenate the original dataframe with the encoded columns

### Rename the dummies for fuel type
de_encoded.rename(columns={'ft_alternative fuel':'alternative_fuel', 'ft_diesel':'diesel', 'ft_diesel_phev':'diesel_phev', 'ft_petrol':'petrol', 'ft_petrol_phev':'petrol_phev'}, inplace=True)

### Linear Regression
import statsmodels.api as sm
import patsy
import numpy as np
# Create the formula with interaction and higher-order terms
formula = 'nedc ~ weight + power + weight:power + diesel + diesel_phev + petrol + petrol_phev + e_year_2011 + e_year_2012 + e_year_2013 + e_year_2014 + e_year_2015 + e_year_2016 + e_year_2017 + e_year_2018 + e_year_2019 + e_year_2020'
Y, X = patsy.dmatrices(formula, data=de_encoded)   # Generate the design matrices
X = sm.add_constant(X)   # add constant
model = sm.OLS(Y, X).fit()  # Fit the linear regression model
print(model.summary())  # Print the model summary

                            OLS Regression Results                            
Dep. Variable:                      y   R-squared:                       0.714
Model:                            OLS   Adj. R-squared:                  0.714
Method:                 Least Squares   F-statistic:                 9.557e+05
Date:                Tue, 08 Oct 2024   Prob (F-statistic):               0.00
Time:                        18:40:28   Log-Likelihood:            -2.7779e+07
No. Observations:             6522187   AIC:                         5.556e+07
Df Residuals:                 6522169   BIC:                         5.556e+07
Df Model:                          17                                         
Covariance Type:            nonrobust                                         
                 coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------
const         75.4534      0.569    132.608      0.0

In [100]:
#df_sales.shape   #(260410, 569)
#df_sales[df_sales['Price. (local)']=='Unspecified'].shape # 14550 rows (14550, 569)
#df_sales[df_sales['Price. (EUR)']=='Unspecified'].shape # 14550 rows   (14550, 569)

(260410, 569)

In [97]:
#df_sales['Price. (local)']
df_sales['Price. (EUR)'].unique()

array(['    31678', '    25510', '    61122', ..., '     9372',
       '    99288', '   107511'], dtype=object)

# 2. Transform from short to long. (20s)

In [3]:
### Drop useless columns
df_s = df_sales.drop(columns=['Make (World)','Model Group','Model (World)','Registration Type','IHS vehicle type','Global Sales Segment',
                 'Global Sales Sub-Segment','Global Sales Price Class','Length class Segmentation','Standard Segmentation',
                 'GI Global Segment','Source descr.','No. of Seats','No. of Cylinders','No. of Gears',
                 'Price. (USD)','Start of Price list','End of Price list','Width', 'Length','Height','Axle Configuration',
                 'No. of Doors','Driven Wheels','Body Type', 'Sub Model Short','Model'])

### Rename Columns
df_s.columns = df_s.columns.str.lower()    # convert column names to lower case
df_s = df_s.rename(columns={'make group': 'firm'})        # treat parent company as firm
df_s = df_s.rename(columns={'price. (local)': 'price_local'})     
df_s = df_s.rename(columns={'price. (eur)': 'price_eur'})          
df_s = df_s.rename(columns={'fuel type': 'ft'})
df_s = df_s.rename(columns={'generation year': 'g_year'})
df_s = df_s.rename(columns={'gross vehicle weight ': 'weight'})
df_s = df_s.rename(columns={'engine (ccm)': 'power'})
df_s = df_s.rename(columns={'engine (hp)': 'horsepower'})
df_s = df_s.rename(columns={'make': 'brand'})
df_s = df_s.rename(columns={'sub model': 'model'})


### Aggregate monthly sales to anual sales from 1981 to 2023
for year in range(1981, 2024):
    month_columns = [f"{year}/{month:02d}" for month in range(1, 13)]  # this create a list of column names for the 12 months of the year, useful skill!
    df_s[str(year)] = df_s[month_columns].sum(axis=1, skipna=True)         # Sum the monthly values for each year, handling missing columns
df_s.drop(columns=df_s.filter(like='/'), inplace=True)                     # Drop the original monthly columns        # columns=df_s.filter(like='/') pick the columns that contain '/', useful skill!

### transform the dataframe from wide to long format
df_s_long = df_s.melt(id_vars=['country', 'firm', 'brand', 'model', 'body group', 'version std', 'trim', 'ft',
       'transmission', 'turbo', 'usa-class', 'engine (ltr)', 'power',
       'engine (kw)', 'horsepower', 'weight', 'g_year', 'wheelbase', 'price_local', 'price_eur'], value_vars=df_s.loc[:, '1981':'2023'].columns.tolist(), var_name='year', value_name='sales') 
df_s_long['year'] = df_s_long['year'].astype(int) # convert 'year' to integer


In [4]:
#############################################################         Data cleaning           #############################################################   
df_s_long = df_s_long[df_s_long['sales']>0]     # 1631064
df_s_long = df_s_long[df_s_long['weight']>0]    #  1596899     # lost  34,165 sales 
df_s_long = df_s_long[df_s_long['price_local']!='Unspecified']    # all price are specified given conditions, no further lost of sales

#### remove unusual fuel types cars
df_s_long['ft'] = df_s_long['ft'].str.lower()
df_s_long = df_s_long[df_s_long['ft']!='lpg (petr.gas)']  
df_s_long = df_s_long[df_s_long['ft']!='natural gas']  
df_s_long = df_s_long[df_s_long['ft']!='fuel cell']
df_s_long = df_s_long[(df_s_long['ft'] != ' 7-sit') & (df_s_long['ft'] != ' 5-sitzer/2wd')] 

### remove luxury cars
df_s_long['price_local'] = df_s_long['price_local'].astype(int)  
df_s_long = df_s_long[df_s_long['price_local']<100000]

### remove missing values in weight, power, horsepower and fuel type.
df_s_long = df_s_long.dropna(subset=['weight', 'power', 'horsepower', 'ft'])    # no missing values in 'weight', 'power', 'horsepower' and 'ft'

### remove low sales
df_s_long = df_s_long[df_s_long['sales']>10]

###########################################################     experiment on 2020 year    #############################################################
# df = df_s_long[df_s_long['year']==2020].copy()

# df.shape[0]                     # 10259 trims
# df[df['sales']>10].shape[0]     # 7309 trims left
# df[df['sales']>100].shape[0]    # 3069 trims left

# df['sales'].sum()                   # 1593993
# df[df['sales']>10]['sales'].sum()   # 1581990/1631064 = 96.9% of sales left
# df[df['sales']>100]['sales'].sum()  # 1404810/1631064 = 86.1% of sales left

In [237]:

#df_s_long[ (df_s_long['year'] == 2020) & (df_s_long['price']<200000) ]['sales'].sum()  # 1,596,439    # loss further 4,460 sales
#df_s_long[ (df_s_long['year'] == 2020) & (df_s_long['price']<150000) ]['sales'].sum()  # 1,594,833    # loss further 1,606 sales
#df_s_long[ (df_s_long['year'] == 2020) & (df_s_long['price']<125000) ]['sales'].sum()  # 1,592,579    # loss further 2,254 sales
#df_s_long[ (df_s_long['year'] == 2020) & (df_s_long['price']<100000) ]['sales'].sum()  # 1,585,935    # loss further 6,644 sales       # 1,585,935/1,631,064 = 97.2% of the total sales in 2020

#df.shape[0]  # conditional on 2020 year, there are 10323 trims
#df[df['price']<150000].shape[0]  # 10277   # loss further 46 trims
#df[df['price']<100000].shape[0]  # 10123   # loss further 154 trims

10259

# 3. Predict emissions

In [ ]:
### Rename fuel type for the purpose of prediction
df_s_long['ft'] = df_s_long['ft'].replace('hev series pet','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('hev/petr.','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('hev/dsl.','diesel')
df_s_long['ft'] = df_s_long['ft'].replace('hev/petr.plugin','petrol_phev')
df_s_long['ft'] = df_s_long['ft'].replace('hev/dsl.plugin','diesel_phev')
df_s_long['ft'] = df_s_long['ft'].replace('electric w/orex','bev')
df_s_long['ft'] = df_s_long['ft'].replace('electric w. rex','petrol_phev')
df_s_long['ft'] = df_s_long['ft'].replace('mev/petrol','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('mev/diesel','diesel')
df_s_long['ft'] = df_s_long['ft'].replace('ethanol-petrol','petrol')


### Add dummies for fuel type and year
df_dummies = pd.get_dummies(df_s_long[['ft','g_year']], columns=['ft', 'g_year'], drop_first=False)   # One-hot encode the columns 'ft' and 'year'
df_dummies = df_dummies.astype(int)     # Convert the encoded columns to integer type, i.e. 0/1 instead of True/False
df_encoded = pd.concat([df_s_long, df_dummies], axis=1)    # Concatenate the original dataframe with the encoded columns

### Generate 'nedc'
df_s_long['nedc'] = model.params[0] \
+ model.params[1]*df_encoded['weight'] \
+ model.params[2]*df_encoded['power'] \
+ model.params[3]*df_encoded['weight']*df_encoded['power'] \
+ model.params[4]*df_encoded['ft_diesel'] \
+ model.params[5]*df_encoded['ft_diesel_phev'] \
+ model.params[6]*df_encoded['ft_petrol'] \
+ model.params[7]*df_encoded['ft_petrol_phev'] \
#+ model.params[8]*df_encoded['g_year_2011'] \
#+ model.params[9]*df_encoded['g_year_2012'] \
#+ model.params[10]*df_encoded['g_year_2013'] \
#+ model.params[11]*df_encoded['g_year_2014'] \
+ model.params[12]*df_encoded['g_year_2015'] \
+ model.params[13]*df_encoded['g_year_2016'] \
+ model.params[14]*df_encoded['g_year_2017'] \
+ model.params[15]*df_encoded['g_year_2018'] \
+ model.params[16]*df_encoded['g_year_2019'] \
+ model.params[17]*df_encoded['g_year_2020']
df_s_long.loc[df_s_long['ft'].isin(['bev', 'electric']), 'nedc'] = 0  # assign 0 to 'nedc' with 'ft' being 'bev' or 'electric'

# 4. Aggregate trims

In [5]:
### rename fuel type for prediction
df_s_long['ft'] = df_s_long['ft'].replace('hev series pet','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('hev/petr.','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('hev/dsl.','diesel')
df_s_long['ft'] = df_s_long['ft'].replace('hev/petr.plugin','petrol_phev')
df_s_long['ft'] = df_s_long['ft'].replace('hev/dsl.plugin','diesel_phev')
df_s_long['ft'] = df_s_long['ft'].replace('electric w/orex','bev')
df_s_long['ft'] = df_s_long['ft'].replace('electric w. rex','petrol_phev')
df_s_long['ft'] = df_s_long['ft'].replace('mev/petrol','petrol')
df_s_long['ft'] = df_s_long['ft'].replace('mev/diesel','diesel')
df_s_long['ft'] = df_s_long['ft'].replace('ethanol-petrol','petrol')
# further rename fuel type for BLP
df_s_long['ft'] = df_s_long['ft'].replace('diesel','icev')
df_s_long['ft'] = df_s_long['ft'].replace('petrol','icev')
df_s_long['ft'] = df_s_long['ft'].replace('petrol_phev','phev')
df_s_long['ft'] = df_s_long['ft'].replace('diesel_phev','phev')
df_s_long['ft'].unique()

### Generate a new variable 'model name' that provides more detailed information of the modelm
df_s_long['engine_variant'] = df_s_long['model'] + ' ' + df_s_long['engine (ltr)'].astype(str) + ' ' + df_s_long['ft']


### add exchange rate column
df_s_long['price_eur'] = df_s_long['price_eur'].astype(int)
df_s_long['ex_rate'] = df_s_long['price_eur']/df_s_long['price_local'] 


### Aggregate the data into model and fuel type level
df_l = df_s_long.groupby(['country','firm', 'brand', 'model', 'engine_variant', 'ft', 'year']).agg({
    'horsepower': 'mean',
    'weight': 'mean', 
    #'nedc': 'mean', 
    'price_local': 'mean', 
    'price_eur':'mean', 
    'ex_rate':'mean',                                                           # for other characteristics, take the mean of different variants of the same model
    'sales': 'sum'     # for sales data, sum the sales across different variants of the same model
}).round(2).copy()
df_l = df_l.reset_index()   # Reset the index to make the groupby columns back to columns

# 5. Adding market size and charging device data (5s)

In [6]:
### clean the household number data
df_size_cp = df_size.iloc[:3,:].copy()   # keep only the first 3 rows of the dataframe
df_size_cp = df_size_cp.melt(id_vars=['country'], value_vars = df_size_cp.columns[1:], var_name='year', value_name='market size')    # transform the df_size to the long format

### match the market size with the 'country' and 'year'
df_l['year'] = df_l['year'].astype(str)   # convert 'year' to 'str' type temporarily
df_l['market size'] = df_l.apply(lambda row: df_size_cp[(df_size_cp['country'] == row['country']) & (df_size_cp['year'] == row['year'])]['market size'].values[0] if not df_size_cp[(df_size_cp['country'] == row['country']) & (df_size_cp['year'] == row['year'])]['market size'].empty else None, axis=1)  # promt: add a column 'market size' to df_long, whose values such that the values of 'market size' match the values in dataframe df_size according to the 'country' and 'year'
df_l['market size'] = df_l['market size'].str.replace(',' , '').astype(float)    # change the data type of 'market size' from object to float
df_l['year'] = df_l['year'].astype(int)   #convert 'year' back to 'int' type

### add charing devices data
# transform df_charge from long to wide format
df_charging_cp = df_charging.pivot(index=['country','year'], columns='charging_type', values='value').reset_index().copy()
df_l['s_charge'] = df_l.apply(lambda row: df_charging_cp[(df_charging_cp['year'] == row['year'])]['s_charge'].values[0] if not df_charging_cp[(df_charging_cp['year'] == row['year'])]['s_charge'].empty else None, axis=1) 
df_l['f_charge'] = df_l.apply(lambda row: df_charging_cp[(df_charging_cp['year'] == row['year'])]['f_charge'].values[0] if not df_charging_cp[(df_charging_cp['year'] == row['year'])]['f_charge'].empty else None, axis=1) 

### add gdp/capita data
#df_gdp.drop(columns = ["Unnamed: 4", "Unnamed: 5", "Unnamed: 6", "Unnamed: 7", "Unnamed: 8"], inplace = True)
df_l['gdp_pc'] = df_l.apply(lambda row: df_gdp[(df_gdp['country'] == row['country']) & (df_gdp['year'] == row['year'])]['gdp_pc_dollar'].values[0] if not df_gdp[(df_gdp['country'] == row['country']) & (df_gdp['year'] == row['year'])]['gdp_pc_dollar'].empty else None, axis=1)

### add EV stock data
df_evstock = df_evstock[df_evstock['region'].isin(['United Kingdom','France','Germany'])]
df_evstock = df_evstock[df_evstock['parameter']=='EV stock']
df_evstock.drop(columns=['category', 'parameter' ,'mode', 'unit'],inplace=True)
df_evstock = df_evstock[df_evstock['powertrain']!='FCEV']
df_evstock_group = df_evstock.groupby(['region','year']).agg({'value':'sum'}).reset_index()
df_evstock_group.rename(columns={'value':'ev_stock'}, inplace=True)
df_l['ev_stock'] = df_l.apply(lambda row: df_evstock_group[(df_evstock_group['region'] == row['country']) & (df_evstock_group['year'] == row['year'])]['ev_stock'].values[0] if not df_evstock_group[(df_evstock_group['region'] == row['country']) & (df_evstock_group['year'] == row['year'])]['ev_stock'].empty else None, axis=1)

# 6. Construct subsidized prices

In [7]:
df_l['subsidies'] = 0

### UK (plug-in grant for BEV and PHEV)
#	2015-2018: 4500 for bev and 2500 for phev, with price below £60,000
#	2019-2020: 3000 for bev or phev, with price below £ 50,000
#	2021:      2500 for bev or phev, with price below £35,000
#	2022-2023: 1500 for bev only, price below 32,000

df_l.loc[(df_l['country']=='United Kingdom') & 
         (df_l['year'].isin([2015, 2016, 2017, 2018])) & 
         (df_l['ft'] == 'bev') &
         (df_l['price_local'] < 60000), 'subsidies'] = 4500

df_l.loc[(df_l['country']=='United Kingdom') & 
         (df_l['year'].isin([2015, 2016, 2017, 2018])) & 
         (df_l['ft'] == 'phev') &
         (df_l['price_local'] < 60000), 'subsidies'] = 4500

df_l.loc[(df_l['country']=='United Kingdom') & 
		 (df_l['year'].isin([2019, 2020])) & 
		 (df_l['ft'].isin(['bev','phev'])) &
		 (df_l['price_local']<50000), 'subsidies'] = 3000

df_l.loc[(df_l['country']=='United Kingdom') & 
		 (df_l['year'].isin([2021])) & 
		 (df_l['ft'].isin(['bev','phev'])) &
		 (df_l['price_local']<35000), 'subsidies'] = 2500

df_l.loc[(df_l['country']=='United Kingdom') & 
		 (df_l['year'].isin([2022,2023])) & 
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<32000), 'subsidies'] = 1500


### Adding subsidies for France (bonus écologique for BEV and PHEV)
#	2015-2016:	6300 for BEV, 1000 for PHEV 
#	2017-2019: 	6000 for BEV, 1000 for PHEV 	
#    2020: 6000 for BEV with price <45,000; 3000 for BEV with price 45,000-60,000; 500 for PHEV with emission of 20-60, price 45,000-60,000.
#	2021: 5000 for BEV with price <45,000; 3000 for BEV with price 45,000-60,000; 500 for PHEV with emission of 20-60, price 45,000-60,000.
#	2022-2023: 4000 for BEV with price <45,000; 3000 for BEV with price 45,000-60,000;	500 for PHEV with emission of 20-60, price 45,000-60,000

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2015,2016])) &
		 (df_l['ft'].isin(['bev'])),'subsidies'] = 6300

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2017,2018,2019])) &
		 (df_l['ft'].isin(['bev'])),'subsidies'] = 6000

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2015,2016,2017,2018,2019])) &
		 (df_l['ft'].isin(['phev'])),'subsidies'] = 1000



df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2020])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<45000),'subsidies'] = 6000

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2021])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<45000),'subsidies'] = 5000

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2022,2023])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<45000),'subsidies'] = 4000


df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2020,2021,2022,2023])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']> 45000) &
		 (df_l['price_local'] <= 60000),'subsidies'] = 3000

df_l.loc[(df_l['country']=='France') &
		 (df_l['year'].isin([2020,2021,2022,2023])) &
		 (df_l['ft'].isin(['phev'])) &
		 (df_l['price_local']> 45000) &
		 (df_l['price_local'] <= 60000),'subsidies'] = 500


### Adding subsidies for Germany (Umweltbonus for BEV and PHEV)
#	2015: 0
#	2016-2018: 4000 for BEV with price <40,000; 3000 for PHEV with price <65,000;
#    2019: 6000 for BEV with price <40,000; 5000 for BEV with price 40,000-60,000; 4500 for PHEV with price <65,000;
#	2020-2023: 9000 for BEV with price <40,000; 6750 for PHEVs with price <65,000;
	
df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2016,2017,2018])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<40000),'subsidies'] = 4000

df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2016,2017,2018])) &
		 (df_l['ft'].isin(['phev'])) &
		 (df_l['price_local']<65000),'subsidies'] = 3000



df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2019])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<40000),'subsidies'] = 6000

df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2019])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']>40000) &
		 (df_l['price_local']<60000),'subsidies'] = 5000

df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2019])) &
		 (df_l['ft'].isin(['phev'])) &
		 (df_l['price_local']<65000),'subsidies'] = 4500



df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2020,2021,2022,2023])) &
		 (df_l['ft'].isin(['bev'])) &
		 (df_l['price_local']<40000),'subsidies'] = 9000

df_l.loc[(df_l['country']=='Germany') &
		 (df_l['year'].isin([2020,2021,2022,2023])) &
		 (df_l['ft'].isin(['phev'])) &
		 (df_l['price_local']>40000) &
		 (df_l['price_local']<60000),'subsidies'] = 6750



###### create column for subsidied price and unsubsidied price

df_l['price_local_sub'] = df_l['price_local'] - df_l['subsidies']

df_l['price_sub'] = df_l['price_local_sub'] * df_l['ex_rate']

df_l['price_unsub'] = df_l['price_eur']

############################# match the values in 'df_rate' wiht 'exchange rate' in 'df_l' with 'pound_to_euro' in 'df_rate' according to 'year' in both dataframes #####################################
#df_l['exchange rate'] = df_l.apply(lambda row: df_rate[(df_rate['year'] == row['year'])]['pound_to_euro'].values[0] if not df_rate[(df_rate['year'] == row['year'])]['pound_to_euro'].empty else None, axis=1)
#df_l['price(£)'] = df_l['price'] / df_l['exchange rate']
#df_l['prices'] = df_l['price(£)'] - df_l['subsidies']   # subtract 'subsidies' from 'price'
#########################################################################################################################################################################################################



# 7. Generate demand instruments

In [8]:
### prepare for running pyblp
df_l['year'] = df_l['year'].astype(str)   # convert 'year' to string type
df_l['market_ids'] = df_l['country'] + '_' + df_l['year']   # create a 'market_id' column equal to 'year'
df_l['product_ids'] = df_l['engine_variant']   # create a 'product_id' column equal to 'model name'
df_l = df_l.rename(columns={'firm': 'firm_ids'})   # rename the column name 'firm' to 'firm_ids'

### Generate indicators variables for the categorical variables.
df_long_dummies = pd.get_dummies(df_l['ft'], columns=['ft'], drop_first=False)   # One-hot encode 'ft'
df_long_dummies = df_long_dummies.astype(int)    # Convert the encoded columns to integer type
df_long = pd.concat([df_l, df_long_dummies], axis=1)      # Concatenate the original dataframe with the encoded columns


## create columns 'demand_instruments0', 'demand_instruments1', and so on.
demand_instruments = pyblp.build_blp_instruments(pyblp.Formulation( '1  + horsepower + weight + phev + bev'), df_long)   # don't add 'ft' to the formula since it is categorical variable.
demand_instruments_df = pd.DataFrame(demand_instruments)
demand_instruments_df.columns = [f'demand_instruments{i}' for i in range(demand_instruments_df.shape[1])]  # rename the columns as 'demand_instruments0', 'demand_instruments1', and so on.

## combine the demand_instruments_df and df_l
# Reset index of both dataframes to ensure they align properly
df_long.reset_index(drop=True, inplace=True)
demand_instruments_df.reset_index(drop=True, inplace=True)
# Now concatenate
df_combined = pd.concat([df_long, demand_instruments_df], axis=1)

### convert data type of 'year' and 'market_ids' to integer
df_combined['year'] = df_combined['year'].astype(int)


# 8. Add demographic data (30s)

In [ ]:
###'QUAL_1': degree obtained
# 2022: 405/19657 = 2.1%                     # Not enough response rate this year!
# 2021: 55260/194737 = 28.4%
# 2020: 56375/211914 = 26.6%
# 2019: 61718/262036 = 23.6%
# 2018: 61322/269537 = 22.8%
# 2017: 60327/273868 = 22.0%
# 2016: 60610/280439 = 21.6%
# 2015: 60814/296223 = 20.5%

In [98]:
### create copies of the demographic data
df_dm22_cp = df_dm22.copy() 
df_dm21_cp = df_dm21.copy() 
df_dm20_cp = df_dm20.copy() 
df_dm19_cp = df_dm19.copy()
df_dm18_cp = df_dm18.copy()
df_dm17_cp = df_dm17.copy()
df_dm16_cp = df_dm16.copy()
df_dm15_cp = df_dm15.copy()
list_cp = [df_dm22_cp, df_dm21_cp, df_dm20_cp, df_dm19_cp, df_dm18_cp, df_dm17_cp, df_dm16_cp, df_dm15_cp]

### Select sample based on response to qualification obtained and net weekly pay
## show the selcted sample size for each year
for df_dm in list_cp:           # add [df_dm14, df_dm13, df_dm12, df_dm11] if there are EV charging point data for these years
    df_dm['year'] = df_dm['market_ids'].astype(int)
    nb = df_dm[(df_dm['QUAL_1'] != -9.00) & (~df_dm['NETWK'].isin([-8, -9, -10]))].shape[0]     # selected sample size
    nb_t = df_dm.shape[0]                                                                 # total sample size
    pc = nb / nb_t                                                                     # proportion of selected sample
    year = df_dm['market_ids'].unique()[0]     # to extract the year 'string' for printing
    print(f'In {year}, selected is {nb}, total is {nb_t}. representation is {round(pc * 100, 2)}%')
    df_dm.drop(df_dm[(df_dm['QUAL_1'] == -9.00) | (df_dm['NETWK'].isin([-8, -9, -10]))].index, inplace=True) # remove the observations where 'QUAL_1' is -9.00 or 'NETWK' is -8, -9, -10
    df_dm['weights'] = 1/nb   # create a 'weights' column equal to 1/sample size
    df_dm.rename(columns={'NETWK': 'income', 'AGE': 'age', 'QUAL_1': 'degree'}, inplace=True)  # rename the columns 'NETWK' as 'income', 'AGE' as 'age', 'QUAL_1' as 'degree'


### Combine the demographics data vertically
df_agent = pd.concat(list_cp, axis=0)


### Generate Monte Carlo nodes
# create the initial mc_stack
integration = pyblp.Integration('monte_carlo', size = df_agent[df_agent['year']== 2015]['market_ids'].shape[0])
mc = pyblp.build_integration(integration, 5)
mc_array = mc['nodes']
mc_stack = pd.DataFrame(mc_array, columns=['nodes0', 'nodes1', 'nodes2', 'nodes3', 'nodes4'])
mc_stack = mc_stack.iloc[0:1]
# stack the mc dataframes
for i in range(2015,2023):       
    integration = pyblp.Integration('monte_carlo', size = df_agent[df_agent['year']== i]['market_ids'].shape[0])
    mc = pyblp.build_integration(integration, 5)
    mc = mc['nodes']
    mc = pd.DataFrame(mc, columns=['nodes0', 'nodes1', 'nodes2', 'nodes3', 'nodes4'])
    mc_stack = pd.concat([mc_stack, mc], axis=0)    
# remove the first row of 'mc_stack'
mc_stack = mc_stack.iloc[1:]  
# concatenate 'df_agent' and 'mc_stack' horizontally without matching the index
df_agent = pd.concat([df_agent.reset_index(drop=True), mc_stack.reset_index(drop=True)], axis=1) 


### randomly select 100 rows of the df_agent for each unique 'market_ids'
df_agent = df_agent.groupby('market_ids').apply(lambda x: x.sample(100)).reset_index(drop=True)
df_agent['weights'] = 1/100 


# convert data type of 'year' and 'market_ids' to integer
df_agent['year'] = df_agent['year'].astype(int)
df_agent['market_ids'] = df_agent['market_ids'].astype(int)


In 2022, selected is 4426, total is 223196. representation is 1.98%
In 2021, selected is 58658, total is 249997. representation is 23.46%
In 2020, selected is 60815, total is 268289. representation is 22.67%
In 2019, selected is 78325, total is 323754. representation is 24.19%
In 2018, selected is 82168, total is 330859. representation is 24.83%
In 2017, selected is 84287, total is 334195. representation is 25.22%
In 2016, selected is 82399, total is 341049. representation is 24.16%
In 2015, selected is 90423, total is 357037. representation is 25.33%


In [107]:
product_data.head()

,firm_ids,brand,model,engine_variant,ft,year,horsepower,weight,price,sales,country,market size,s_charge,r_charge,subsidies,prices,market_ids,product_ids,bev,icev,phev,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7,demand_instruments8,demand_instruments9,shares
11,Aston Martin Finance Gr.,Aston Martin,Aston Martin DB11,Aston Martin DB11 4.0 icev,icev,2017,510.0,2.06,144900.0,29,United Kingdom,27226.0,6036.4,1018.6,0,144.90,2017,Aston Martin DB11 4.0 icev,0,1,0,2.0,977.0,4.02,0.0,0.0,731.0,137763.6,1517.0991,27.0,13.0,0.000002
12,Aston Martin Finance Gr.,Aston Martin,Aston Martin DB11,Aston Martin DB11 4.0 icev,icev,2018,510.0,2.06,144900.0,235,United Kingdom,27576.0,8711.6,1544.8,0,144.90,2018,Aston Martin DB11 4.0 icev,0,1,0,0.0,0.0,0.00,0.0,0.0,780.0,151410.5,1628.6975,30.0,13.0,0.000017
13,Aston Martin Finance Gr.,Aston Martin,Aston Martin DB11,Aston Martin DB11 4.0 icev,icev,2019,510.0,2.06,144900.0,193,United Kingdom,27824.0,13448.0,2288.8,0,144.90,2019,Aston Martin DB11 4.0 icev,0,1,0,0.0,0.0,0.00,0.0,0.0,729.0,146415.5,1545.4306,30.0,17.0,0.000014
14,Aston Martin Finance Gr.,Aston Martin,Aston Martin DB11,Aston Martin DB11 4.0 icev,icev,2020,510.0,2.06,144900.0,102,United Kingdom,27893.0,18595.8,3310.4,0,144.90,2020,Aston Martin DB11 4.0 icev,0,1,0,0.0,0.0,0.00,0.0,0.0,683.0,144241.9,1480.2867,64.0,33.0,0.000007
34,Aston Martin Finance Gr.,Aston Martin,Aston Martin DB9,Aston Martin DB9 5.9 icev,icev,2016,551.0,2.19,138950.0,56,United Kingdom,27109.0,4367.4,810.0,0,138.95,2016,Aston Martin DB9 5.9 icev,0,1,0,1.0,426.0,1.83,0.0,0.0,713.0,135334.6,1474.4539,21.0,10.0,0.000004


# 9. Estimation

In [37]:
### Standardization
product_data = df_combined.copy()

# country
#product_data = product_data[product_data['country'].isin(['United Kingdom'])]
# time
product_data = product_data[(product_data['year']>=2015) & (product_data['year']<=2023)]     
# accounting for subsidies
#product_data['prices'] = product_data['price_sub']
product_data['prices'] = product_data['price_sub']/product_data['gdp_pc']
#product_data['prices'] = product_data['price_unsub']    
#product_data['prices'] = product_data['price_unsub']/product_data['gdp_pc']   
# impactful models
product_data = product_data[product_data['sales']>1000]        
# scale
product_data['prices'] = product_data['prices'] / 1000  
product_data['weight'] = product_data['weight'] / 1000
product_data['horsepower'] = product_data['horsepower'] / 100  
product_data['s_charge'] = product_data['s_charge'] / product_data['ev_stock']
product_data['f_charge'] = product_data['f_charge'] / product_data['ev_stock']
# market size determination
market_size_discount = 1
product_data['shares'] = product_data['sales'] / (product_data['market size']*1000*market_size_discount)

# add 'clutering_ids' column
product_data['clustering_ids'] = product_data['model']

# add dummy variables for 'year'
pd_dummies = pd.get_dummies(product_data['year'], columns=['year'], drop_first=False)
pd_dummies = pd_dummies.astype(int)
product_data = pd.concat([product_data, pd_dummies], axis=1)

#Interaction between fuel type and year
product_data['phev_2015'] = product_data['phev']*product_data[2015]
product_data['phev_2016'] = product_data['phev']*product_data[2016]
product_data['phev_2017'] = product_data['phev']*product_data[2017]
product_data['phev_2018'] = product_data['phev']*product_data[2018]
product_data['phev_2019'] = product_data['phev']*product_data[2019]
product_data['phev_2020'] = product_data['phev']*product_data[2020]
product_data['phev_2021'] = product_data['phev']*product_data[2021]
product_data['phev_2022'] = product_data['phev']*product_data[2022]
product_data['phev_2023'] = product_data['phev']*product_data[2023]

product_data['bev_2015'] = product_data['bev']*product_data[2015]
product_data['bev_2016'] = product_data['bev']*product_data[2016]
product_data['bev_2017'] = product_data['bev']*product_data[2017]
product_data['bev_2018'] = product_data['bev']*product_data[2018]
product_data['bev_2019'] = product_data['bev']*product_data[2019]
product_data['bev_2020'] = product_data['bev']*product_data[2020]
product_data['bev_2021'] = product_data['bev']*product_data[2021]
product_data['bev_2022'] = product_data['bev']*product_data[2022]
product_data['bev_2023'] = product_data['bev']*product_data[2023]

### 9.1 Standatd Logit

In [38]:
### Standard Logit (using instruments)
#logit_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev + phev:s_charge + bev:s_charge ', absorb='C(model)')  # with fixed effects
logit_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev + phev:s_charge + bev:s_charge + phev_2016 + phev_2017 + phev_2018 + phev_2019 + phev_2020 + phev_2021 + phev_2022 + phev_2023 + bev_2016 + bev_2017 + bev_2018 + bev_2019 + bev_2020 + bev_2021 + bev_2022 + bev_2023', absorb='C(model)')
### Problem
problem = pyblp.Problem(logit_formulation, product_data)
logit_results = problem.solve()
logit_results

Problem Results Summary:
GMM   Objective  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Shares   Condition Number  Condition Number 
----  ---------  -------  ----------------  -----------------
 2    +1.6E+02      0         +8.3E+13          +1.3E+07     

Cumulative Statistics:
Computation   Objective 
   Time      Evaluations
-----------  -----------
 00:00:00         2     

Beta Estimates (Robust SEs in Parentheses):
  prices      weight    horsepower     phev        bev      phev*s_charge  bev*s_charge  phev_2016   phev_2017   phev_2018   phev_2019   phev_2020   phev_2021   phev_2022   phev_2023    bev_2016    bev_2017    bev_2018    bev_2019    bev_2020    bev_2021    bev_2022    bev_2023 
----------  ----------  ----------  ----------  ----------  -------------  ------------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ----------  ---------

### 9.2. BLP with no demographics

In [30]:
### No demographics BLP
X1_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev + phev:s_charge + bev:s_charge + phev_2016 + phev_2017 + phev_2018 + phev_2019 + phev_2020 + phev_2021 + phev_2022 + phev_2023 + bev_2016 + bev_2017 + bev_2018 + bev_2019 + bev_2020 + bev_2021 + bev_2022 + bev_2023', absorb='C(model)')  # with fixed effects
X2_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev')
product_formulations = (X1_formulation, X2_formulation)
### Initial parameters
initial_sigma = np.diag([0.1,0.1,0.1,0.1,0.1,0.1])
### Bounds for parameters
bounds = (
    [[0,0,0,0,0,0],       # lower bounds
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0]],   
    
    [[None,0,0,0,0,0],     # upper bounds
    [0,None,0,0,0,0],
    [0,0,None,0,0,0],
    [0,0,0,None,0,0],
    [0,0,0,0,None,0],
    [0,0,0,0,0,None]], 
    )
nd_problem = pyblp.Problem(product_formulations, 
                           product_data,  
                           integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed':0}))
nd_results = nd_problem.solve(initial_sigma,
                              sigma_bounds=bounds,
                              optimization=pyblp.Optimization('l-bfgs-b', {'gtol': 1e-5}),  
                              iteration=pyblp.Iteration('squarem', {'atol': 1e-14}),
                              )    # W_type='clustered', se_type='clustered'
nd_results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.6E+02     +2.9E-05        +0.0E+00         +2.0E+02         0         +1.2E+17          +7.0E+08     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:00:32       Yes          67           78          6819         21754   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
  Sigma:        1         prices      weight    horsepower     phev        bev    
----------  ----------  ----------  ----------  ----------

In [55]:
### No demographics BLP
### Initial parameters
initial_sigma = np.diag([0.1,0.1,0.1,0.1,0.1,0.1])
### Bounds for parameters
bounds = (
    [[0,0,0,0,0,0],       # lower bounds
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0],
    [0,0,0,0,0,0]],   
    
    [[None,0,0,0,0,0],     # upper bounds
    [0,None,0,0,0,0],
    [0,0,None,0,0,0],
    [0,0,0,None,0,0],
    [0,0,0,0,None,0],
    [0,0,0,0,0,None]], 
    )
X1_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev + phev:s_charge + phev:year + bev:s_charge + bev:year', absorb='C(model)')  # with fixed effects
X2_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev ')
product_formulations = (X1_formulation, X2_formulation)
nd_problem = pyblp.Problem(product_formulations, 
                           product_data,  
                           integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed':0}))
nd_results = nd_problem.solve(initial_sigma,
                              sigma_bounds=bounds,
                              optimization=pyblp.Optimization('l-bfgs-b', {'gtol': 1e-5}),  
                              iteration=pyblp.Iteration('squarem', {'atol': 1e-14}),
                              )    # W_type='clustered', se_type='clustered'
nd_results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.4E+02     +1.0E-03        +0.0E+00         +5.3E+02         0         +8.7E+12          +3.1E+12     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:01:23       Yes          76           100         74213       225103   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
  Sigma:        1         prices      weight    horsepower     phev        bev    
----------  ----------  ----------  ----------  ----------

In [41]:
### No demographics BLP
X1_formulation = pyblp.Formulation('1 + prices + weight + horsepower + phev + bev + phev:s_charge + bev:s_charge', absorb='C(model)')  # with fixed effects
X2_formulation = pyblp.Formulation('1 + prices + weight + horsepower + bev + phev')
product_formulations = (X1_formulation, X2_formulation)
nd_problem = pyblp.Problem(product_formulations, 
                           product_data,  
                           integration = pyblp.Integration('monte_carlo', size=100, specification_options={'seed':0}))
nd_results = nd_problem.solve(initial_sigma,
                              sigma_bounds=bounds,
                              optimization=pyblp.Optimization('l-bfgs-b', {'gtol': 1e-5}),  
                              iteration=pyblp.Iteration('squarem', {'atol': 1e-14}),
                              W_type='clustered', se_type='clustered')  
nd_results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +4.7E+01     +2.9E-04        +0.0E+00         +1.5E+05         0         +3.1E+12          +2.1E+08     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:02:02       Yes          82           107         94415       286317   

Nonlinear Coefficient Estimates (Robust SEs Adjusted for 468 Clusters in Parentheses):
  Sigma:        1         prices      weight    horsepower     bev         phev   
----------  ----------  --------

In [268]:
df_sales[df_sales['Model'].str.contains('tesla',case=False)]

,Country,Make Group,Make,Make (World),Model Group,Model,Model (World),Sub Model Short,Sub Model,Registration Type,IHS vehicle type,Global Sales Segment,Global Sales Sub-Segment,Global Sales Price Class,Length class Segmentation,Standard Segmentation,GI Global Segment,Body Group,Body Type,Version STD,Trim,Source descr.,Fuel Type,Driven Wheels,No. of Doors,Transmission,Turbo,Axle Configuration,No. of Seats,USA-CLASS,Engine (ltr),Engine (ccm),Engine (kW),Engine (HP),Gross Vehicle Weight,Height,Length,Width,Generation Year,No. of Cylinders,No. of Gears,Wheelbase,Price. (local),Price. (EUR),Price. (USD),Start of Price list,End of Price list,1981/01,1981/02,1981/03,1981/04,1981/05,1981/06,1981/07,1981/08,1981/09,1981/10,1981/11,1981/12,1982/01,1982/02,1982/03,1982/04,1982/05,1982/06,1982/07,1982/08,1982/09,1982/10,1982/11,1982/12,1983/01,1983/02,1983/03,1983/04,1983/05,1983/06,1983/07,1983/08,1983/09,1983/10,1983/11,1983/12,1984/01,1984/02,1984/03,1984/04,1984/05,1984/06,1984/07,1984/08,1984/09,1984/10,1984/11,1984/12,1985/01,1985/02,1985/03,1985/04,1985/05,1985/06,1985/07,1985/08,1985/09,1985/10,1985/11,1985/12,1986/01,1986/02,1986/03,1986/04,1986/05,1986/06,1986/07,1986/08,1986/09,1986/10,1986/11,1986/12,1987/01,1987/02,1987/03,1987/04,1987/05,1987/06,1987/07,1987/08,1987/09,1987/10,1987/11,1987/12,1988/01,1988/02,1988/03,1988/04,1988/05,1988/06,1988/07,1988/08,1988/09,1988/10,1988/11,1988/12,1989/01,1989/02,1989/03,1989/04,1989/05,1989/06,1989/07,1989/08,1989/09,1989/10,1989/11,1989/12,1990/01,1990/02,1990/03,1990/04,1990/05,1990/06,1990/07,1990/08,1990/09,1990/10,1990/11,1990/12,1991/01,1991/02,1991/03,1991/04,1991/05,1991/06,1991/07,1991/08,1991/09,1991/10,1991/11,1991/12,1992/01,1992/02,1992/03,1992/04,1992/05,1992/06,1992/07,1992/08,1992/09,1992/10,1992/11,1992/12,1993/01,1993/02,1993/03,1993/04,1993/05,1993/06,1993/07,1993/08,1993/09,1993/10,1993/11,1993/12,1994/01,1994/02,1994/03,1994/04,1994/05,1994/06,1994/07,1994/08,1994/09,1994/10,1994/11,1994/12,1995/01,1995/02,1995/03,1995/04,1995/05,1995/06,1995/07,1995/08,1995/09,1995/10,1995/11,1995/12,1996/01,1996/02,1996/03,1996/04,1996/05,1996/06,1996/07,1996/08,1996/09,1996/10,1996/11,1996/12,1997/01,1997/02,1997/03,1997/04,1997/05,1997/06,1997/07,1997/08,1997/09,1997/10,1997/11,1997/12,1998/01,1998/02,1998/03,1998/04,1998/05,1998/06,1998/07,1998/08,1998/09,1998/10,1998/11,1998/12,1999/01,1999/02,1999/03,1999/04,1999/05,1999/06,1999/07,1999/08,1999/09,1999/10,1999/11,1999/12,2000/01,2000/02,2000/03,2000/04,2000/05,2000/06,2000/07,2000/08,2000/09,2000/10,2000/11,2000/12,2001/01,2001/02,2001/03,2001/04,2001/05,2001/06,2001/07,2001/08,2001/09,2001/10,2001/11,2001/12,2002/01,2002/02,2002/03,2002/04,2002/05,2002/06,2002/07,2002/08,2002/09,2002/10,2002/11,2002/12,2003/01,2003/02,2003/03,2003/04,2003/05,2003/06,2003/07,2003/08,2003/09,2003/10,2003/11,2003/12,2004/01,2004/02,2004/03,2004/04,2004/05,2004/06,2004/07,2004/08,2004/09,2004/10,2004/11,2004/12,2005/01,2005/02,2005/03,2005/04,2005/05,2005/06,2005/07,2005/08,2005/09,2005/10,2005/11,2005/12,2006/01,2006/02,2006/03,2006/04,2006/05,2006/06,2006/07,2006/08,2006/09,2006/10,2006/11,2006/12,2007/01,2007/02,2007/03,2007/04,2007/05,2007/06,2007/07,2007/08,2007/09,2007/10,2007/11,2007/12,2008/01,2008/02,2008/03,2008/04,2008/05,2008/06,2008/07,2008/08,2008/09,2008/10,2008/11,2008/12,2009/01,2009/02,2009/03,2009/04,2009/05,2009/06,2009/07,2009/08,2009/09,2009/10,2009/11,2009/12,2010/01,2010/02,2010/03,2010/04,2010/05,2010/06,2010/07,2010/08,2010/09,2010/10,2010/11,2010/12,2011/01,2011/02,2011/03,2011/04,2011/05,2011/06,2011/07,2011/08,2011/09,2011/10,2011/11,2011/12,2012/01,2012/02,2012/03,2012/04,2012/05,2012/06,2012/07,2012/08,2012/09,2012/10,2012/11,2012/12,2013/01,2013/02,2013/03,2013/04,2013/05,2013/06,2013/07,2013/08,2013/09,2013/10,2013/11,2013/12,2014/01,2014/02,2014/03,2014/04,2014/05,2014/06,2014/07,2014/08,2014/09,2014/10,2014/11,2014/12,2015/01,2015/02,2015/03,2015/04,2015/05,2015/06,2015/07,2015/08,2015/09,2015/10,2015/11,2015

### 9.3. BLP with demographics

In [102]:
### Data
agent_data = df_agent[(df_agent['year']>=2015)&(df_agent['year']<=2021)].copy()
product_data = df_combined[(df_combined['year']>=2015) & (df_combined['year']<=2021)].copy()
product_data['prices'] = product_data['price_sub']
#product_data['prices'] = product_data['price_unsub']

market_size_discount = 1
product_data['shares'] = product_data['sales'] / (product_data['market size']*1000*market_size_discount)
#product_data = product_data.drop(columns=['demand_instruments3', 'demand_instruments4','demand_instruments8','demand_instruments9'])  # drop some instruments

### With demographics BLP
X1_formulation = pyblp.Formulation('1 + prices + weight + horsepower  + phev + bev + phev:s_charge + bev:s_charge', absorb='C(model)')  
X2_formulation = pyblp.Formulation('1 + prices + horsepower + bev + phev')
product_formulations = (X1_formulation, X2_formulation)
agent_formulation = pyblp.Formulation('0 + income + degree')
### Optimization method
opt = pyblp.Optimization('l-bfgs-b', {'gtol':1e-4})
### Initial parameters
initial_sigma = np.diag([1,1,1,0,0])
initial_pi = np.array([
    [0, 0],
    [1, 0],
    [0, 0],
    [0, 1],
    [0, 1],
    ])
### Bounds for parameters
bounds = (
    [[0.1,0,0,0,0],       # lower bounds
    [0,0.1,0,0,0],
    [0,0,0.1,0,0],
    [0,0,0,0,0],
    [0,0,0,0,0]],   

    [[10,0,0,0,0],     # upper bounds
    [0,10,0,0,0],
    [0,0,10,0,0],
    [0,0,0,10,0],
    [0,0,0,0,10]], 
    )

wd_problem = pyblp.Problem(product_formulations, product_data, agent_formulation, agent_data)
wd_results = wd_problem.solve(initial_sigma, initial_pi, optimization=opt, sigma_bounds = bounds)   # restrict the random coefficientmatrix to be diagonal
wd_results

Problem Results Summary:
GMM   Objective    Projected    Reduced Hessian  Reduced Hessian  Clipped  Weighting Matrix  Covariance Matrix
Step    Value    Gradient Norm  Min Eigenvalue   Max Eigenvalue   Shares   Condition Number  Condition Number 
----  ---------  -------------  ---------------  ---------------  -------  ----------------  -----------------
 2    +1.4E+03     +9.0E+04        -1.2E+00         +2.1E+08         0         +1.1E+10          +2.3E+12     

Cumulative Statistics:
Computation  Optimizer  Optimization   Objective   Fixed Point  Contraction
   Time      Converged   Iterations   Evaluations  Iterations   Evaluations
-----------  ---------  ------------  -----------  -----------  -----------
 00:21:27       Yes          3            29         314406       943594   

Nonlinear Coefficient Estimates (Robust SEs in Parentheses):
  Sigma:        1         prices    horsepower    bev       phev    |     Pi:        income      degree  
----------  ----------  ---------- 

In [199]:
product_data[(product_data['country']=='United Kingdom') &
             (product_data['brand'].str.contains('porsche',case=False))] #.shape[0]  # 1,000

,country,firm_ids,brand,model,engine_variant,ft,year,horsepower,weight,price_local,price_eur,ex_rate,sales,market size,s_charge,f_charge,subsidies,price_local_sub,price_sub,price_unsub,market_ids,product_ids,bev,icev,phev,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7,demand_instruments8,demand_instruments9,prices,shares
52305,United Kingdom,Volkswagen,Porsche,Porsche 718 Boxster,Porsche 718 Boxster 2.0 icev,icev,2016,299.00,1670.00,42700.00,50070.50,1.17,574,27109.0,19000.0,1000.0,0,42700.00,49959.0000,50070.50,United Kingdom_2016,Porsche 718 Boxster 2.0 icev,0,1,0,147.0,30439.82,302013.30,6.0,1.0,551.0,97040.44,1136030.33,14.0,9.0,49959.0000,0.000021
52306,United Kingdom,Volkswagen,Porsche,Porsche 718 Boxster,Porsche 718 Boxster 2.0 icev,icev,2017,299.00,1670.00,44229.00,51863.50,1.17,791,27226.0,20000.0,1000.0,0,44229.00,51747.9300,51863.50,United Kingdom_2017,Porsche 718 Boxster 2.0 icev,0,1,0,150.0,29357.74,309174.28,6.0,1.0,563.0,98872.27,1161985.81,19.0,12.0,51747.9300,0.000029
52307,United Kingdom,Volkswagen,Porsche,Porsche 718 Boxster,Porsche 718 Boxster 2.0 icev,icev,2018,299.00,1675.00,46483.67,54507.33,1.17,659,27576.0,23000.0,1700.0,0,46483.67,54385.8939,54507.33,United Kingdom_2018,Porsche 718 Boxster 2.0 icev,0,1,0,157.0,30143.78,324682.92,6.0,1.0,598.0,108083.39,1240306.17,22.0,12.0,54385.8939,0.000024
52308,United Kingdom,Volkswagen,Porsche,Porsche 718 Boxster,Porsche 718 Boxster 2.0 icev,icev,2019,299.00,1677.50,51680.25,60600.75,1.17,647,27824.0,28000.0,2000.0,0,51680.25,60465.8925,60600.75,United Kingdom_2019,Porsche 718 Boxster 2.0 icev,0,1,0,146.0,29008.40,306875.73,4.0,2.0,559.0,104975.45,1175467.32,23.0,15.0,60465.8925,0.000023
52309,United Kingdom,Volkswagen,Porsche,Porsche 718 Boxster,Porsche 718 Boxster 2.0 icev,icev,2020,299.00,1679.00,51405.20,60278.40,1.17,250,27893.0,31000.0,3300.0,0,51405.20,60144.0840,60278.40,United Kingdom_2020,Porsche 718 Boxster 2.0 icev,0,1,0,150.0,32901.49,326598.41,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,60144.0840,0.000009
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
52646,United Kingdom,Volkswagen,Porsche,Porsche Panamera,Porsche Panamera 4.8 icev,icev,2016,441.00,2500.00,93391.00,109511.00,1.17,31,27109.0,19000.0,1000.0,0,93391.00,109267.4700,109511.00,United Kingdom_2016,Porsche Panamera 4.8 icev,0,1,0,147.0,30297.82,301183.30,6.0,1.0,551.0,97040.44,1136030.33,14.0,9.0,109267.4700,0.000001
52647,United Kingdom,Volkswagen,Porsche,Porsche Taycan,Porsche Taycan 0.0 bev,bev,2020,435.00,2880.00,83367.00,97757.00,1.17,1973,27893.0,31000.0,3300.0,0,83367.00,97539.3900,97757.00,United Kingdom_2020,Porsche Taycan 0.0 bev,1,0,0,150.0,32765.49,325397.41,13.0,7.0,511.0,99369.81,1094885.72,46.0,25.0,97539.3900,0.000071
52648,United Kingdom,Volkswagen,Porsche,Porsche Taycan,Porsche Taycan 0.0 bev,bev,2021,414.50,2860.62,81513.38,95583.50,1.17,3212,28119.0,50000.0,4500.0,0,81513.38,95370.6546,95583.50,United Kingdom_2021,Porsche Taycan 0.0 bev,1,0,0,154.0,34903.26,339514.40,21.0,11.0,483.0,94652.95,1050039.99,55.0,38.0,95370.6546,0.000114
52649,United Kingdom,Volkswagen,Porsche,Porsche Taycan,Porsche Taycan 0.0 bev,bev,2022,406.57,2838.48,82548.70,96797.61,1.17,3996,28243.0,74000.0,9700.0,0,82548.70,96581.9790,96797.61,United Kingdom_2022,Porsche Taycan 0.0 bev,1,0,0,146.0,32967.77,326696.38,20.0,13.0,445.0,88104.18,979536.82,58.0,50.0,96581.9790,0.000141


0

In [231]:
product_data[(product_data['country']=='United Kingdom') &
             (product_data['year']==2020) &   (product_data['sales']>1000)]

,country,firm_ids,brand,model,engine_variant,ft,year,horsepower,weight,price_local,price_eur,ex_rate,sales,market size,s_charge,f_charge,subsidies,price_local_sub,price_sub,price_unsub,market_ids,product_ids,bev,icev,phev,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7,demand_instruments8,demand_instruments9,prices,shares
35994,United Kingdom,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2020,130.07,1888.97,27048.79,31717.59,1.17,18445,27893.0,31000.0,3300.0,0,27048.79,31647.0843,31717.59,United Kingdom_2020,BMW 1-Series 1.5 icev,0,1,0,65.0,16855.85,149508.61,9.0,2.0,596.0,115584.38,1271765.55,50.0,31.0,31647.0843,0.000661
36024,United Kingdom,BMW,BMW,BMW 1-Series,BMW 1-Series 2.0 icev,icev,2020,189.19,2011.67,31610.95,37067.33,1.17,6468,27893.0,31000.0,3300.0,0,31610.95,36984.8115,37067.33,United Kingdom_2020,BMW 1-Series 2.0 icev,0,1,0,65.0,16796.73,149385.91,9.0,2.0,596.0,115584.38,1271765.55,50.0,31.0,36984.8115,0.000232
36048,United Kingdom,BMW,BMW,BMW 2-Series,BMW 2-Series 1.5 icev,icev,2020,136.00,1923.89,30524.17,35792.94,1.17,1762,27893.0,31000.0,3300.0,0,30524.17,35713.2789,35792.94,United Kingdom_2020,BMW 2-Series 1.5 icev,0,1,0,65.0,16849.92,149473.69,9.0,2.0,596.0,115584.38,1271765.55,50.0,31.0,35713.2789,0.000063
36066,United Kingdom,BMW,BMW,BMW 2-Series,BMW 2-Series 3.0 icev,icev,2020,379.40,2015.50,49964.00,58588.30,1.17,1893,27893.0,31000.0,3300.0,0,49964.00,58457.8800,58588.30,United Kingdom_2020,BMW 2-Series 3.0 icev,0,1,0,65.0,16606.52,149382.08,9.0,2.0,596.0,115584.38,1271765.55,50.0,31.0,58457.8800,0.000068
36096,United Kingdom,BMW,BMW,BMW 2-Series Gran Coupe,BMW 2-Series Gran Coupe 1.5 icev,icev,2020,140.00,1894.09,28019.55,32855.91,1.17,2616,27893.0,31000.0,3300.0,0,28019.55,32782.8735,32855.91,United Kingdom_2020,BMW 2-Series Gran Coupe 1.5 icev,0,1,0,65.0,16845.92,149503.49,9.0,2.0,596.0,115584.38,1271765.55,50.0,31.0,32782.8735,0.000094
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54550,United Kingdom,Volkswagen,Volkswagen,Volkswagen Tiguan,Volkswagen Tiguan 1.5 icev,icev,2020,147.15,2144.00,31355.00,36767.20,1.17,10667,27893.0,31000.0,3300.0,0,31355.00,36685.3500,36767.20,United Kingdom_2020,Volkswagen Tiguan 1.5 icev,0,1,0,150.0,33053.34,326133.41,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,36685.3500,0.000382
54566,United Kingdom,Volkswagen,Volkswagen,Volkswagen Tiguan,Volkswagen Tiguan 2.0 icev,icev,2020,170.96,2269.80,37118.57,43525.57,1.17,15074,27893.0,31000.0,3300.0,0,37118.57,43428.7269,43525.57,United Kingdom_2020,Volkswagen Tiguan 2.0 icev,0,1,0,150.0,33029.53,326007.61,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,43428.7269,0.000540
54594,United Kingdom,Volkswagen,Volkswagen,Volkswagen Touareg,Volkswagen Touareg 3.0 icev,icev,2020,272.60,2836.00,56363.00,66091.72,1.17,2225,27893.0,31000.0,3300.0,0,56363.00,65944.7100,66091.72,United Kingdom_2020,Volkswagen Touareg 3.0 icev,0,1,0,150.0,32927.89,325441.41,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,65944.7100,0.000080
54640,United Kingdom,Volkswagen,Volkswagen,Volkswagen Touran,Volkswagen Touran 1.5 icev,icev,2020,150.00,2135.00,30609.64,35893.14,1.17,1605,27893.0,31000.0,3300.0,0,30609.64,35813.2788,35893.14,United Kingdom_2020,Volkswagen Touran 1.5 icev,0,1,0,150.0,33050.49,326142.41,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,35813.2788,0.000058


In [226]:
product_data[product_data['sales']>1000]

,country,firm_ids,brand,model,engine_variant,ft,year,horsepower,weight,price_local,price_eur,ex_rate,sales,market size,s_charge,f_charge,subsidies,price_local_sub,price_sub,price_unsub,market_ids,product_ids,bev,icev,phev,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7,demand_instruments8,demand_instruments9,prices,shares
27,France,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2015,113.80,1890.29,29569.57,29569.57,1.00,3533,28961.4,9900.0,580.0,0,29569.57,29569.5700,29569.57,France_2015,BMW 1-Series 1.5 icev,0,1,0,50.0,10405.35,108368.99,2.0,1.0,579.0,84802.11,1160431.43,7.0,13.0,29569.5700,0.000122
28,France,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2016,115.30,1888.33,29264.11,29264.11,1.00,6041,29173.4,19000.0,1000.0,0,29264.11,29264.1100,29264.11,France_2016,BMW 1-Series 1.5 icev,0,1,0,53.0,11128.01,115389.20,4.0,1.0,583.0,89235.04,1169694.18,12.0,14.0,29264.1100,0.000207
29,France,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2017,113.04,1883.93,29503.93,29503.93,1.00,7006,29267.7,20000.0,1000.0,0,29503.93,29503.9300,29503.93,France_2017,BMW 1-Series 1.5 icev,0,1,0,49.0,10651.05,108194.85,6.0,1.0,617.0,94471.78,1233755.63,17.0,15.0,29503.9300,0.000239
30,France,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2018,116.03,1899.54,31580.26,31580.26,1.00,6903,29778.5,23000.0,1700.0,0,31580.26,31580.2600,31580.26,France_2018,BMW 1-Series 1.5 icev,0,1,0,50.0,10884.00,111110.26,7.0,1.0,629.0,95220.82,1263930.42,19.0,17.0,31580.2600,0.000232
31,France,BMW,BMW,BMW 1-Series,BMW 1-Series 1.5 icev,icev,2019,122.65,1902.23,31822.30,31822.30,1.00,5296,30048.2,28000.0,2000.0,0,31822.30,31822.3000,31822.30,France_2019,BMW 1-Series 1.5 icev,0,1,0,51.0,11161.32,113689.71,4.0,1.0,582.0,94777.28,1193332.69,21.0,21.0,31822.3000,0.000176
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
54719,United Kingdom,Volkswagen,Volkswagen,Volkswagen Up!,Volkswagen Up! 1.0 icev,icev,2019,74.06,1343.06,12133.37,14227.71,1.17,8372,27824.0,28000.0,2000.0,0,12133.37,14196.0429,14227.71,United Kingdom_2019,Volkswagen Up! 1.0 icev,0,1,0,146.0,29233.34,307210.17,4.0,2.0,559.0,104975.45,1175467.32,23.0,15.0,14196.0429,0.000301
54720,United Kingdom,Volkswagen,Volkswagen,Volkswagen Up!,Volkswagen Up! 1.0 icev,icev,2020,72.45,1344.50,13529.00,15864.30,1.17,3272,27893.0,31000.0,3300.0,0,13529.00,15828.9300,15864.30,United Kingdom_2020,Volkswagen Up! 1.0 icev,0,1,0,150.0,33128.04,326932.91,13.0,8.0,511.0,99369.81,1094885.72,46.0,25.0,15828.9300,0.000117
54721,United Kingdom,Volkswagen,Volkswagen,Volkswagen Up!,Volkswagen Up! 1.0 icev,icev,2021,78.00,1347.74,14539.52,17049.06,1.17,3110,28119.0,50000.0,4500.0,0,14539.52,17011.2384,17049.06,United Kingdom_2021,Volkswagen Up! 1.0 icev,0,1,0,154.0,35239.76,341027.28,21.0,12.0,483.0,94652.95,1050039.99,55.0,38.0,17011.2384,0.000111
54722,United Kingdom,Volkswagen,Volkswagen,Volkswagen Up!,Volkswagen Up! 1.0 icev,icev,2022,79.57,1350.00,15502.62,18178.48,1.17,2423,28243.0,74000.0,9700.0,0,15502.62,18138.0654,18178.48,United Kingdom_2022,Volkswagen Up! 1.0 icev,0,1,0,146.0,33294.77,328184.86,20.0,14.0,445.0,88104.18,979536.82,58.0,50.0,18138.0654,0.000086


In [269]:
df_emission = pd.read_csv('/Users/simon/Library/CloudStorage/OneDrive-UniversityCollegeLondon/Cars data/Official fuel efficiency/data/2010-2023_fe_uk.csv',low_memory=False) 

In [427]:
df = df_emission.copy()
# total car registration in uk in 2020 is 1620502
uk = df[(df['year']==2020)  &   (df['country']=='GB')].drop(columns=['id']).copy()
# replace missing values with -9999 in all columns of 'uk'
uk.fillna(-9999, inplace=True)
uk['duplicates'] = uk.groupby(['country', 'vfn', 'mp', 'mh', 'man', 'mms', 'tan', 't', 'va', 've',
                                'mk', 'cn', 'ct', 'cr'
                                ]).transform('size')

uk.drop_duplicates(subset=['country', 'vfn', 'mp', 'mh', 'man', 'mms', 'tan', 't', 'va', 've',
                            'mk', 'cn', 'ct', 'cr'
                            ], inplace=True)


In [430]:
uk['tan'].nunique()  # 1

2053

In [431]:
uk['vfn'].nunique()

3116

In [432]:
uk['cn'].nunique()

4600

In [433]:
uk[uk['duplicates']>1]['cn'].nunique()  # 1

4158

In [436]:
uk[uk['duplicates']==1].shape[0]  # 0

6778

In [434]:
uk

,country,vfn,mp,mh,man,mms,tan,t,va,ve,mk,cn,ct,cr,r,m (kg),mt,enedc (g/km),ewltp (g/km),w (mm),at1 (mm),at2 (mm),ft,fm,ec (cm3),ep (kw),z (wh/km),it,ernedc (g/km),erwltp (g/km),de,vf,status,year,date of registration,fuel consumption,ech,rlfi,electric range (km),duplicates
5073598,GB,IP-08_BBC_0098_00-W0V-1,PSA-OPEL,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,VAUXHALL,E4*2007/46*0996*20,B-K,CA028DD12,BW1BBCKH315,VAUXHALL,ASTRA SRI VX LINE NAV TURBO,M1,M1,1,1280.0,1394.0,99.0,123.0,2662.0,1548.0,1565.0,petrol,M,1199.0,-9999.0,-9999.0,"E4 17, E4 19",2.4,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,666
5073599,GB,IP-HMR____MA5_1318-W0V-0,PSA-OPEL,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,VAUXHALL,E4*2007/46*1194*10,P7 MONOCAB C,EA024DB12,BB1VANPDTA5,VAUXHALL,CROSSLAND X ELITE,M1,M1,1,1165.0,1310.0,106.0,142.0,2604.0,1513.0,1491.0,petrol,M,1199.0,-9999.0,-9999.0,"E2 19, E2 17",2.3,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,811
5073600,GB,IP-HNS____MB6F3423-W0V-0,PSA-OPEL,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,VAUXHALL,E2*2007/46*0597*14,Z,R,HNSK-C1P400,VAUXHALL,GRANDLAND X ELITE NAV TURBO,M1,M1,1,1395.0,1561.0,114.0,147.0,2675.0,1579.0,1587.0,petrol,M,1199.0,-9999.0,-9999.0,E2 17 19,2.5,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1157
5073601,GB,IP-HNS____AT6_1430-W0V-0,PSA-OPEL,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,VAUXHALL,E4*2007/46*1194*10,P7 MONOCAB C,EA023DB12,BB1VABPAZA5,VAUXHALL,CROSSLAND X GRIFFIN TURBO AUTO,M1,M1,1,1280.0,1427.0,113.0,146.0,2604.0,1513.0,1491.0,petrol,M,1199.0,-9999.0,-9999.0,"E2 19, E2 17",2.4,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,345
5073603,GB,IP-HNS____AT6_1430-W0V-0,PSA-OPEL,OPEL AUTOMOBILE,OPEL AUTOMOBILE GMBH,VAUXHALL,E4*2007/46*1194*10,P7 MONOCAB C,EA023DB12,BB1VABPAZA5,VAUXHALL,CROSSLAND X ELITE NAV TURBO A,M1,M1,1,1280.0,1427.0,113.0,146.0,2604.0,1513.0,1491.0,petrol,M,1199.0,-9999.0,-9999.0,"E2 19, E2 17",2.4,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1889
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
6693052,GB,IP-MQB27ZZ_A2_0221-VSS-1,VW-SAIC,SEAT,SEAT SA,SEAT,e9*2007/46*3134*27,KJ,BDKRF,FM6SFM6AJ0244BF3A1CAD,SEAT,IBIZA FR SPORT TSI,M1,M1,1,1134.0,1305.0,107.0,131.0,2548.0,1499.0,1479.0,petrol,M,999.0,-9999.0,-9999.0,e9 17 19,2.6,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1
6693301,GB,IP-09-VSS-2018-0055,VW-SAIC,SEAT,SEAT SA,SEAT,e9*2007/46*3134*16,KJ,XDKRF,FM6SFM6AJ0167CP3A1BA,SEAT,ARONA FR SPORT TSI,M1,M1,1,1189.0,1300.0,114.0,130.0,2552.0,1503.0,1486.0,petrol,M,999.0,-9999.0,-9999.0,-9999,-9999.0,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1
6693429,GB,IP-MQB37AS_A1_0305-VSS-1,VW-SAIC,SEAT,SEAT SA,SEAT,e9*2007/46*6666*11,KN,XDPCAX0,FM6FM62Q0364BFML751CAD,SEAT,TARRACO XCELLENCE LUX TSI EVO,M1,M1,1,1634.0,1823.0,137.0,171.0,2787.0,1575.0,1564.0,petrol,M,1498.0,-9999.0,-9999.0,e9 17 19,2.7,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1
6693606,GB,IP-09-VSS-2018-0078,VW-SAIC,SEAT,SEAT SA,SEAT,e9*2007/46*0094*34,5F,BDACAX0V,FM6SFM6AJ0157CPVLO51CA,SEAT,LEON SE DYNAMIC TSI EVO,M1,M1,1,1239.0,1374.0,113.0,139.0,2618.0,1533.0,1504.0,petrol,M,1498.0,-9999.0,-9999.0,-9999,-9999.0,-9999.0,-9999.0,-9999.0,F,2020,-9999.0,-9999.0,-9999.0,-9999.0,-9999.0,1


In [437]:
product_data[product_data['model'].str.contains('crossland',case=False)]

,country,firm_ids,brand,model,engine_variant,ft,year,horsepower,weight,price_local,price_eur,ex_rate,sales,market size,s_charge,f_charge,subsidies,price_local_sub,price_sub,price_unsub,market_ids,product_ids,bev,icev,phev,demand_instruments0,demand_instruments1,demand_instruments2,demand_instruments3,demand_instruments4,demand_instruments5,demand_instruments6,demand_instruments7,demand_instruments8,demand_instruments9,prices,r_charge,shares,clustering_ids
47957,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2017,1.0523,1.75886,18638.64,21855.95,1.17,2023,27226.0,20.0,1000.0,0,18638.64,21807.2088,21855.95,United Kingdom_2017,Vauxhall Crossland/X 1.2 icev,0,1,0,124.0,17342.01,244862.25,0.0,1.0,589.0,111081.77,1226208.98,25.0,12.0,21.807209,1.0,0.000074,Vauxhall Crossland/X
47958,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2018,1.1023,1.77177,19620.48,23007.19,1.17,10256,27576.0,23.0,1700.0,0,19620.48,22955.9616,23007.19,United Kingdom_2018,Vauxhall Crossland/X 1.2 icev,0,1,0,143.0,20472.68,282685.14,0.0,0.0,612.0,117943.26,1282207.18,28.0,13.0,22.955962,1.7,0.000372,Vauxhall Crossland/X
47959,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2019,1.1373,1.77280,20534.75,24079.27,1.17,15503,27824.0,28.0,2000.0,0,20534.75,24025.6575,24079.27,United Kingdom_2019,Vauxhall Crossland/X 1.2 icev,0,1,0,118.0,17633.07,233771.68,0.0,0.0,587.0,116536.05,1248476.07,27.0,17.0,24.025657,2.0,0.000557,Vauxhall Crossland/X
47960,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2020,1.1075,1.75591,21068.18,24704.77,1.17,17744,27893.0,31.0,3300.0,0,21068.18,24649.7706,24704.77,United Kingdom_2020,Vauxhall Crossland/X 1.2 icev,0,1,0,110.0,17028.94,222392.62,6.0,5.0,551.0,115430.61,1199014.60,53.0,28.0,24.649771,3.3,0.000636,Vauxhall Crossland/X
47961,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2021,1.1193,1.76531,23045.68,27023.60,1.17,15398,28119.0,50.0,4500.0,0,23045.68,26963.4456,27023.60,United Kingdom_2021,Vauxhall Crossland/X 1.2 icev,0,1,0,105.0,15221.37,211198.68,7.0,11.0,532.0,114637.41,1179451.02,69.0,39.0,26.963446,4.5,0.000548,Vauxhall Crossland/X
47962,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2022,1.1260,1.77450,23432.83,27477.57,1.17,11063,28243.0,74.0,9700.0,0,23432.83,27416.4111,27477.57,United Kingdom_2022,Vauxhall Crossland/X 1.2 icev,0,1,0,100.0,14735.45,203583.62,10.0,12.0,491.0,106630.47,1103713.56,68.0,52.0,27.416411,9.7,0.000392,Vauxhall Crossland/X
47963,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.2 icev,icev,2023,1.2125,1.80321,25747.64,30191.89,1.17,6964,28358.0,98.0,20000.0,0,25747.64,30124.7388,30191.89,United Kingdom_2023,Vauxhall Crossland/X 1.2 icev,0,1,0,90.0,13249.11,185918.74,13.0,15.0,471.0,103408.18,1064708.18,67.0,60.0,30.124739,20.0,0.000246,Vauxhall Crossland/X
47965,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.5 icev,icev,2019,1.0538,1.82656,21374.38,25063.69,1.17,1064,27824.0,28.0,2000.0,0,21374.38,25008.0246,25063.69,United Kingdom_2019,Vauxhall Crossland/X 1.5 icev,0,1,0,118.0,17641.42,233717.92,0.0,0.0,587.0,116536.05,1248476.07,27.0,17.0,25.008025,2.0,0.000038,Vauxhall Crossland/X
47968,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.5 icev,icev,2022,1.1200,1.82700,25129.50,29467.10,1.17,1589,28243.0,74.0,9700.0,0,25129.50,29401.5150,29467.10,United Kingdom_2022,Vauxhall Crossland/X 1.5 icev,0,1,0,100.0,14736.05,203531.12,10.0,12.0,491.0,106630.47,1103713.56,68.0,52.0,29.401515,9.7,0.000056,Vauxhall Crossland/X
47971,United Kingdom,STELLANTIS,Vauxhall,Vauxhall Crossland/X,Vauxhall Crossland/X 1.6 icev,icev,2018,1.0688,1.82750,20315.62,23822.44,1.17,2477,27576.0,23.0,1700.0,0,20315.62,23769.2754,23822.44,United Kingdom_2018,Vauxhall Crosslan